# Instructions

This colab contains code used to run the ridgeplot visualization for the Concurrent Trait Shaping experiments on the various LLMs experimented on in the paper "Personality Traits in Large Language Models" (https://arxiv.org/pdf/2307.00184). The code requires "scored" output of the LLM inference dataframes - which are themselves the output of the Concurrent Trait Shaping Analysis notebook. The code assumes that all the data produced and consumed in the colab lives in a local filesystem either in a cloud instance running a Jupyter notebook such as Google Colab or a desktop. But those file I/O operations can easily be replaced to use any other file management solutions.

To run this colab:
1. Connect to an appropriate runtime. (For instance, if running the bulk inference directly from the colab, connect to a GPU kernel.)
2. Check experiment parameters below.
3. Run the code cells for generating the ridge plot visualization.

# Setup


In [ ]:
#@title Install visualization dependencies
%pip install plotly
%pip install matplotlib
%pip install pandas
%pip install scipy

In [ ]:
#@title Load Dependencies
from typing import List, OrderedDict, Union
import plotly.graph_objs as go
from plotly.subplots import make_subplots

import itertools
import pandas as pd
import numpy as np

import scipy

In [ ]:
SPID = ['item_preamble_id',
        'item_postamble_id',
        'response_scale_id',
        'response_choice_postamble_id',
        'model_id']

BFI_SCALE_IDS = ['BFI-EXT', 'BFI-AGR', 'BFI-CON', 'BFI-NEU', 'BFI-OPE']
IPIP_SCALE_IDS = ['IPIP300-EXT', 'IPIP300-AGR', 'IPIP300-CON', 'IPIP300-NEU', 'IPIP300-OPE']
VALIDATION_SCALE_IDS = ['PA', 'NA', 'CSE', 'CPI', 'PHYS', 'VRBL', 'ANGR', 'HSTL', 'ACHV', 'CONF', 'SCRT']

# Ridgeplots

In [ ]:
#@title Helper Functions
def get_domain_fragments(big5_id):
  """Returns list of preamble ID fragments for one domain."""
  return [f'{big5_id}{i}' for i in range(1, 10)]


DIMENSION_PREFIXES = ['ext', 'agr', 'con', 'neu', 'ope']

def get_big5_lvl_fragments():
  """Returns list of preamble ID fragments for all Big Five domains."""
  big5_id_fragments = DIMENSION_PREFIXES
  nested_fragments = [get_domain_fragments(big5_id) for big5_id in big5_id_fragments]
  preamble_id_fragments = list(itertools.chain(*nested_fragments))
  return preamble_id_fragments


def subset_one_preamble(input_df, id_fragment):
  return input_df[input_df['item_preamble_id'].str.contains(id_fragment)][IPIP_SCALE_IDS]


def subset_by_preambles(input_df, id_fragments):
  """Subsets data by a given list of item preamble fragments."""
  preambles = []

  for id_fragment in id_fragments:
    preambles.append(subset_one_preamble(input_df, id_fragment))

  return pd.concat(preambles, keys=id_fragments)


def describe_by_preambles(input_df, id_fragments,
                          by: Union[str, List[str]] = ['median', 'min', 'max', 'std']):
  # organize data by preamble_id fragment
  df_by_preambles = subset_by_preambles(input_df, id_fragments)

  # group by preamble_id fragments
  df_grouped = df_by_preambles.groupby(level=0)

  # aggregate by specified summary stats
  summary = df_grouped.agg(by)

  return summary

##Plot for Concurrent Shaping
The code below reads in all the scored dataframes for all the models to be compared. Each row of the ridgeplot corresponds to a specific model's score distribution, and each column represents a specific personality dimension. The variable `MODEL_NAMES` contains the names of the models that are compared and that will be rendered on the plot for each row. The variable `models` is a list of tuples, where the first entry in the tuple is a model id (used later on in the code) and the second entry is the path to the scored df in the local filesystem.

In [ ]:
#@markdown Update MODEL_NAMES and models vars based on your models being compared.
MODEL_NAMES = ['Llama 2 7B', 'Llama 2 13B', 'Mistral 7B', 'Mixtral 8x7B', 'GPT 3.5 Turbo', 'GPT 4o mini', 'GPT 4o']
models = [('ll_7b', 'results_abl03-ds_Llama-2-7b-chat-hf_combined.pkl.scored_df'),
          ('ll_13b', 'results_abl03-ds_Llama-2-13b-chat-hf_combined.pkl.scored_df'),
          ('mistral_7b', 'results_abl03-dp-Mistral-7B-Instruct-v0.1_combined.pkl.scored_df'),
          ('mixtral_8x7b', 'results_abl03-dp-Mixtral-8x7B-Instruct-v0.1_combined.pkl.scored_df'),
          ('gpt_35turbo', 'results_abl03-sf_gpt-3.5-turbo-0125_combined.pkl.scored_df'),
          ('gpt_4omini', 'results_abl03-dp_gpt-4o-mini_combined.pkl.scored_df'),
          ('gpt_4o', 'results_abl03-dp_gpt-4o_combined.pkl.scored_df')]
all_df = []
for (model_id, drive_filepath) in models:
  df = pd.read_pickle(drive_filepath)
  df['model_id'] = model_id
  all_df.append(df)

PLOT_SPACE = np.linspace(1., 5.1)
PLOT_COLUMNS = ['IPIP-NEO EXT', 'IPIP-NEO AGR', 'IPIP-NEO CON', 'IPIP-NEO NEU', 'IPIP-NEO OPE']

fig = make_subplots(rows=len(MODEL_NAMES), cols=len(PLOT_COLUMNS),
                    shared_xaxes=True, shared_yaxes=True,
                    column_titles=PLOT_COLUMNS,
                    row_titles=MODEL_NAMES,
                    x_title='Observed Personality Scores',
                    y_title='Frequency Distribution of Response Scores',
                    vertical_spacing=0.02, horizontal_spacing=0.01)
big5_domain_lvls = get_big5_lvl_fragments()

ALL_TRAITS = OrderedDict({
    'ext': 'IPIP300-EXT',
    'neu': 'IPIP300-NEU',
    'ope': 'IPIP300-OPE',
    'agr': 'IPIP300-AGR',
    'con': 'IPIP300-CON',
})

for row, df in enumerate(all_df):
  for col, (trait, plot_col) in enumerate(ALL_TRAITS.items()):
    for i, l in enumerate([1, 9]):
      level = f'{trait}{l}'
      sub_df = df[df.item_preamble_id.str.contains(level)]
      dist_to_plot = sub_df[plot_col]
      counts, bins, _ = scipy.stats.binned_statistic(dist_to_plot.values, values=None, statistic='count', bins=PLOT_SPACE, range=(1., 5.1))
      scatter_plot = go.Scatter(
          x=np.concatenate([np.array([1.]), bins]),
          y=np.concatenate([np.array([1.]), counts]),
          name=['Prompted Extremely Low', 'Prompted Extremely High'][i],
          line=dict(color=['red', 'blue'][i]),
          fill='toself',
          mode='lines+text')
      fig.add_trace(scatter_plot, row=row+1, col=col+1)

fig.update_layout(width=1024, height=2048, showlegend=True)
fig.update_yaxes(title='', visible=True, showticklabels=False)


yaxis_labels = ['yaxis', 'yaxis6', 'yaxis11', 'yaxis16', 'yaxis21', 'yaxis26', 'yaxis31', 'yaxis36', 'yaxis41', 'yaxis46', 'yaxis51', 'yaxis56']
for axis_id in yaxis_labels[:len(models)]:
  fig['layout'][axis_id].update({
      'showticklabels': True, 'visible': True,
      'tickmode': 'array',
      'tickfont': dict(size=16)
  })
for axis_id in ['xaxis16', 'xaxis17', 'xaxis18', 'xaxis19', 'xaxis20']:
  fig['layout'][axis_id].update({
      'tickfont': dict(size=16)
  })
for fig_legend in fig['data'][2:]:
  fig_legend['showlegend'] = False
fig.update_layout(title_x=0.5)
fig.update_layout(legend=dict(
    orientation='h',
    yanchor='bottom',
    y=1.03,
    xanchor='right',
    x=1,
    font=dict(size=22)
))
fig.for_each_annotation(lambda a: a.update(font=dict(size=(22 if a['textangle'] != 90 else 16))))

fig.show()